<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey_Model_Mark_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#Import Even-Strength On-Ice Totals Data
skater_EV_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/EV%20On-Ice%20Totals.csv')
print(skater_EV_totals_raw)
skater_EV_totals_raw.head()

                 Player Season Team Position  GP      TOI    GF%    SF%  \
0            A.J. Greer  21-22  N.J        L   9    70.45  50.50  57.90   
1          Aaron Ekblad  19-20  FLA        D  67  1220.55  56.75  54.15   
2          Aaron Ekblad  20-21  FLA        D  35   603.07  55.98  55.00   
3          Aaron Ekblad  21-22  FLA        D  61  1047.80  63.35  58.86   
4            Aaron Ness  19-20  ARI        D  24   289.15  49.44  51.48   
...                 ...    ...  ...      ...  ..      ...    ...    ...   
2650        Zdeno Chara  19-20  BOS        D  68  1166.42  62.20  49.01   
2651        Zdeno Chara  20-21  WSH        D  55   813.33  51.49  49.44   
2652        Zdeno Chara  21-22  NYI        D  72  1191.52  53.26  44.96   
2653  Zemgus Girgensons  19-20  BUF        C  69   783.73  45.67  47.49   
2654  Zemgus Girgensons  21-22  BUF      C/L  56   668.30  44.00  47.21   

        FF%    CF%  ...       CA    xGF    xGA     G±      S±      F±      C±  \
0     57.33  56.89

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,54.34,3.16,2.22,0.06,10.82,14.68,17.36,0.94,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,54.15,54.03,53.68,...,1014.02,49.88,48.26,16.49,105.40,133.71,161.08,1.62,10.08,90.93
2,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.00,54.02,54.30,...,456.49,24.44,19.71,4.76,57.29,62.33,85.88,4.73,7.07,93.20
3,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,58.86,59.73,59.25,...,832.83,57.52,39.95,28.16,206.60,294.26,378.31,17.57,9.74,91.94
4,Aaron Ness,19-20,ARI,D,24,289.15,49.44,51.48,49.81,49.65,...,274.00,10.80,10.43,-0.29,8.70,-1.58,-3.86,0.37,8.45,90.83


In [5]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_totals = skater_EV_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,3.16,2.22
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,49.88,48.26
2,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,24.44,19.71
3,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,57.52,39.95
4,Aaron Ness,19-20,ARI,D,24,289.15,49.44,50.87,12.82,13.11,10.80,10.43
...,...,...,...,...,...,...,...,...,...,...,...,...
2650,Zdeno Chara,19-20,BOS,D,68,1166.42,62.20,50.51,57.18,34.75,41.95,41.11
2651,Zdeno Chara,20-21,WSH,D,55,813.33,51.49,51.79,35.05,33.02,28.36,26.40
2652,Zdeno Chara,21-22,NYI,D,72,1191.52,53.26,46.08,48.37,42.45,46.88,54.86
2653,Zemgus Girgensons,19-20,BUF,C,69,783.73,45.67,49.54,22.91,27.25,25.39,25.86


In [6]:
#Add columns for G+/-,xG+/-
skater_EV_totals['G+/-'] = skater_EV_totals['GF'] - skater_EV_totals['GA']
skater_EV_totals['xG+/-'] = skater_EV_totals['xGF'] - skater_EV_totals['xGA']
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,58.74,3.01,2.95,3.16,2.22,0.06,0.94
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,56.75,50.83,69.36,52.87,49.88,48.26,16.49,1.62
2,Aaron Ekblad,20-21,FLA,D,35,603.07,55.98,55.36,22.29,17.53,24.44,19.71,4.76,4.73
3,Aaron Ekblad,21-22,FLA,D,61,1047.80,63.35,59.01,66.82,38.66,57.52,39.95,28.16,17.57
4,Aaron Ness,19-20,ARI,D,24,289.15,49.44,50.87,12.82,13.11,10.80,10.43,-0.29,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,Zdeno Chara,19-20,BOS,D,68,1166.42,62.20,50.51,57.18,34.75,41.95,41.11,22.43,0.84
2651,Zdeno Chara,20-21,WSH,D,55,813.33,51.49,51.79,35.05,33.02,28.36,26.40,2.03,1.96
2652,Zdeno Chara,21-22,NYI,D,72,1191.52,53.26,46.08,48.37,42.45,46.88,54.86,5.92,-7.98
2653,Zemgus Girgensons,19-20,BUF,C,69,783.73,45.67,49.54,22.91,27.25,25.39,25.86,-4.34,-0.47


In [7]:
#Import PP On-Ice Rates Data
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Totals.csv')
print(skater_PP_totals_raw)
skater_PP_totals_raw.head()

                 Player Season Team Position  GP     TOI     GF%    SF%  \
0          Aaron Ekblad  19-20  FLA        D  67   95.90   92.22  81.87   
1          Aaron Ekblad  20-21  FLA        D  35  132.75   91.82  87.33   
2          Aaron Ekblad  21-22  FLA        D  61  214.78   92.51  86.16   
3          Adam Boqvist  19-20  CHI        D  41   87.02   88.20  81.06   
4          Adam Boqvist  20-21  CHI        D  35  118.65   82.03  75.97   
...                 ...    ...  ...      ...  ..     ...     ...    ...   
1303      Zach Werenski  19-20  CBJ        D  63  171.17   80.98  84.22   
1304      Zach Werenski  20-21  CBJ        D  33   73.60  100.00  81.71   
1305      Zach Werenski  21-22  CBJ        D  64  215.48   87.27  79.95   
1306       Zack Kassian  19-20  EDM        R  58   23.18   54.38  91.63   
1307  Zemgus Girgensons  21-22  BUF      C/L  55   54.37  100.00  83.74   

        FF%    CF%  ...     CA    xGF   xGA     G±      S±      F±      C±  \
0     84.52  85.63  .

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,81.87,84.52,85.63,...,22.50,9.81,0.72,9.44,63.83,86.56,111.57,9.09,12.57,95.21
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,87.33,87.48,87.89,...,33.55,18.23,1.53,19.73,132.80,175.41,209.85,16.70,13.94,91.44
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,86.16,86.54,87.19,...,51.02,29.39,2.64,32.58,171.71,231.62,296.13,26.75,17.33,91.27
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,81.06,84.35,86.69,...,19.11,8.91,0.91,6.41,53.41,75.83,105.34,8.00,10.62,93.92
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,75.97,79.96,82.55,...,32.46,11.76,3.35,10.52,56.65,84.80,121.14,8.41,16.26,88.74


In [8]:
#Reduce PP On-Ice Totals Data
skater_PP_totals = skater_PP_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,93.16,10.31,0.87,9.81,0.72
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,92.26,21.66,1.93,18.23,1.53
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,91.76,35.45,2.87,29.39,2.64
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,90.73,7.40,0.99,8.91,0.91
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,77.83,13.47,2.95,11.76,3.35
...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,87.14,17.46,4.10,17.75,2.62
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,85.02,4.99,0.00,5.56,0.98
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,81.96,25.51,3.72,24.45,5.38
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,96.83,1.18,0.99,1.22,0.04


In [9]:
#Add columns for G+/-,xG+/-,G/s,xG/s
skater_PP_totals['G+/-'] = skater_PP_totals['GF'] - skater_PP_totals['GA']
skater_PP_totals['xG+/-'] = skater_PP_totals['xGF'] - skater_PP_totals['xGA']
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,93.16,10.31,0.87,9.81,0.72,9.44,9.09
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,92.26,21.66,1.93,18.23,1.53,19.73,16.70
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,91.76,35.45,2.87,29.39,2.64,32.58,26.75
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,90.73,7.40,0.99,8.91,0.91,6.41,8.00
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,77.83,13.47,2.95,11.76,3.35,10.52,8.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,87.14,17.46,4.10,17.75,2.62,13.36,15.13
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,85.02,4.99,0.00,5.56,0.98,4.99,4.58
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,81.96,25.51,3.72,24.45,5.38,21.79,19.07
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,96.83,1.18,0.99,1.22,0.04,0.19,1.18


In [10]:
#Add column for mean G+/- and xG+/-
G_mean = skater_PP_totals['G+/-'].mean()
xG_mean = skater_PP_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [11]:
#Add column for stats above average for G+/- and xG+/-
skater_PP_totals['GAA'] = skater_PP_totals['G+/-'] - G_mean
skater_PP_totals['xGAA'] = skater_PP_totals['xG+/-'] - xG_mean
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-,GAA,xGAA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,93.16,10.31,0.87,9.81,0.72,9.44,9.09,-2.792102,-2.726284
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,92.26,21.66,1.93,18.23,1.53,19.73,16.70,7.497898,4.883716
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,91.76,35.45,2.87,29.39,2.64,32.58,26.75,20.347898,14.933716
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,90.73,7.40,0.99,8.91,0.91,6.41,8.00,-5.822102,-3.816284
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,77.83,13.47,2.95,11.76,3.35,10.52,8.41,-1.712102,-3.406284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,87.14,17.46,4.10,17.75,2.62,13.36,15.13,1.127898,3.313716
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,85.02,4.99,0.00,5.56,0.98,4.99,4.58,-7.242102,-7.236284
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,81.96,25.51,3.72,24.45,5.38,21.79,19.07,9.557898,7.253716
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,96.83,1.18,0.99,1.22,0.04,0.19,1.18,-12.042102,-10.636284


In [12]:
#Drop columns
skater_PP_totals = skater_PP_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA'])
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GF%,GF,GA,G+/-,GAA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,10.31,0.87,9.44,-2.792102
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,21.66,1.93,19.73,7.497898
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,35.45,2.87,32.58,20.347898
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,7.40,0.99,6.41,-5.822102
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,13.47,2.95,10.52,-1.712102
...,...,...,...,...,...,...,...,...,...,...,...
1303,Zach Werenski,19-20,CBJ,D,63,171.17,80.98,17.46,4.10,13.36,1.127898
1304,Zach Werenski,20-21,CBJ,D,33,73.60,100.00,4.99,0.00,4.99,-7.242102
1305,Zach Werenski,21-22,CBJ,D,64,215.48,87.27,25.51,3.72,21.79,9.557898
1306,Zack Kassian,19-20,EDM,R,58,23.18,54.38,1.18,0.99,0.19,-12.042102


In [14]:
#Import SH On-Ice Totals Data
skater_SH_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/SH%20On-Ice%20Totals.csv')
#print(skater_SH_totals_raw)
skater_SH_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,22.51,21.41,19.08,...,227.77,3.86,19.19,-22.86,-98.13,-129.47,-174.07,-15.33,4.98,82.03
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,17.92,14.84,15.32,...,131.66,1.30,10.00,-7.00,-64.49,-93.88,-107.84,-8.70,11.15,89.08
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.05,17.45,16.22,...,257.22,4.56,19.13,-11.29,-122.07,-158.45,-207.42,-14.57,15.97,89.17
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,20.18,17.25,15.49,...,233.65,2.48,17.19,-10.41,-87.56,-141.39,-190.84,-14.71,10.26,88.52
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,11.66,11.33,9.96,...,276.05,1.65,20.36,-11.45,-136.27,-187.12,-245.50,-18.71,14.77,90.76


In [15]:
#Reduce SH On-Ice Totals Data
skater_SH_totals = skater_SH_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,16.75,2.00,24.86,3.86,19.19
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,11.50,2.01,9.01,1.30,10.00
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.25,6.00,17.29,4.56,19.13
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,12.61,3.04,13.45,2.48,17.19
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,7.50,3.06,14.51,1.65,20.36
...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,10.96,0.00,21.23,2.88,23.40
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,4.66,0.99,17.14,0.79,16.18
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,8.65,0.93,6.52,1.19,12.57
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,11.86,1.18,23.88,2.24,16.65


In [16]:
#Add columns for G+/-,xG+/-
skater_SH_totals['G+/-'] = skater_SH_totals['GF'] - skater_SH_totals['GA']
skater_SH_totals['xG+/-'] = skater_SH_totals['xGF'] - skater_SH_totals['xGA']
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,16.75,2.00,24.86,3.86,19.19,-22.86,-15.33
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,11.50,2.01,9.01,1.30,10.00,-7.00,-8.70
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.25,6.00,17.29,4.56,19.13,-11.29,-14.57
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,12.61,3.04,13.45,2.48,17.19,-10.41,-14.71
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,7.50,3.06,14.51,1.65,20.36,-11.45,-18.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,10.96,0.00,21.23,2.88,23.40,-21.23,-20.52
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,4.66,0.99,17.14,0.79,16.18,-16.15,-15.39
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,8.65,0.93,6.52,1.19,12.57,-5.59,-11.38
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,11.86,1.18,23.88,2.24,16.65,-22.70,-14.41


In [ ]:
#Add column for mean G+/- and xG+/-
G_mean = skater_SH_totals['G+/-'].mean()
xG_mean = skater_SH_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [17]:
#Add column for stats above average for G+/- and xG+/-
skater_SH_totals['GAA'] = skater_SH_totals['G+/-'] - G_mean
skater_SH_totals['xGAA'] = skater_SH_totals['xG+/-'] - xG_mean
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,xGF%,GF,GA,xGF,xGA,G+/-,xG+/-,GAA,xGAA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,16.75,2.00,24.86,3.86,19.19,-22.86,-15.33,-35.092102,-27.146284
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,11.50,2.01,9.01,1.30,10.00,-7.00,-8.70,-19.232102,-20.516284
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.25,6.00,17.29,4.56,19.13,-11.29,-14.57,-23.522102,-26.386284
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,12.61,3.04,13.45,2.48,17.19,-10.41,-14.71,-22.642102,-26.526284
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,7.50,3.06,14.51,1.65,20.36,-11.45,-18.71,-23.682102,-30.526284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,10.96,0.00,21.23,2.88,23.40,-21.23,-20.52,-33.462102,-32.336284
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,4.66,0.99,17.14,0.79,16.18,-16.15,-15.39,-28.382102,-27.206284
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,8.65,0.93,6.52,1.19,12.57,-5.59,-11.38,-17.822102,-23.196284
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,11.86,1.18,23.88,2.24,16.65,-22.70,-14.41,-34.932102,-26.226284


In [18]:
skater_SH_totals = skater_SH_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA'])
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GF%,GF,GA,G+/-,GAA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,2.00,24.86,-22.86,-35.092102
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,2.01,9.01,-7.00,-19.232102
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,6.00,17.29,-11.29,-23.522102
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,3.04,13.45,-10.41,-22.642102
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,3.06,14.51,-11.45,-23.682102
...,...,...,...,...,...,...,...,...,...,...,...
1303,Zdeno Chara,19-20,BOS,D,67,214.07,0.00,0.00,21.23,-21.23,-33.462102
1304,Zdeno Chara,20-21,WSH,D,55,145.72,5.46,0.99,17.14,-16.15,-28.382102
1305,Zdeno Chara,21-22,NYI,D,68,123.65,12.48,0.93,6.52,-5.59,-17.822102
1306,Zemgus Girgensons,19-20,BUF,C,68,147.87,4.71,1.18,23.88,-22.70,-34.932102


In [19]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Stats%20Box.csv')
#print(skater_box_totals_raw)
skater_box_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,...,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2,...,20,11,1,6,0,0,5,2,1,1
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,...,34,56,14,10,0,0,-4,0,0,0
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,...,24,46,14,8,1,1,-6,0,0,0
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,...,62,54,13,9,0,0,-4,0,0,0
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,...,17,49,0,5,0,0,5,0,0,0


In [20]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2','Points']]
skater_box_totals.head()

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1


In [44]:
#Look at positions to get correct F/D average points
#print(skater_box_totals['Position'].unique())
#print(skater_box_totals.count())

forwards_table = skater_box_totals.loc[(skater_box_totals.Position == "L") | (skater_box_totals.Position == "C") | (skater_box_totals.Position == "C/L") | (skater_box_totals.Position == "R") | (skater_box_totals.Position == "L/R") | (skater_box_totals.Position == "C/R")]
forwards_table.head(10)
#print(forwards_table.count())

defensemen_table = skater_box_totals.loc[(skater_box_totals.Position == "D") | (skater_box_totals.Position == "D/L")]
defensemen_table.head(10)
#print(defensemen_table.count())

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,PAA
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,26.890075
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,7.890075
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,42.890075
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,-13.109925
5,Adam Boqvist,19-20,CHI,D,41,665.12,4,5,4,13,-1.109925
6,Adam Boqvist,20-21,CHI,D,35,594.65,2,2,12,16,1.890075
7,Adam Boqvist,21-22,CBJ,D,52,886.05,11,4,7,22,7.890075
15,Adam Fox,19-20,NYR,D,70,1323.30,8,20,14,42,27.890075
16,Adam Fox,20-21,NYR,D,55,1358.85,5,19,23,47,32.890075
17,Adam Fox,21-22,NYR,D,78,1862.67,11,33,30,74,59.890075


In [27]:
#Get Mean of Points
Points_mean = skater_box_totals['Points'].mean()
#print(Points_mean)

Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)

Defensemen_mean = defensemen_table['Points'].mean()
#print(Defensemen_mean)

In [28]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine

forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
forwards_table.head(10)

defensemen_table['PAA'] = defensemen_table['Points'] - Defensemen_mean
defensemen_table.head(10)


#Add dataframes back together and sort again

concat_frames = [forwards_table, defensemen_table]
skater_box_totals = pd.concat(concat_frames)
skater_box_totals = skater_box_totals.sort_index()
skater_box_totals.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,PAA
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2,-20.633276
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,26.890075
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,7.890075
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,42.890075
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,-13.109925
5,Adam Boqvist,19-20,CHI,D,41,665.12,4,5,4,13,-1.109925
6,Adam Boqvist,20-21,CHI,D,35,594.65,2,2,12,16,1.890075
7,Adam Boqvist,21-22,CBJ,D,52,886.05,11,4,7,22,7.890075
8,Adam Brooks,19-20,TOR,C,7,54.97,0,2,1,3,-19.633276
9,Adam Brooks,20-21,TOR,C,11,117.45,4,0,1,5,-17.633276


In [29]:
combined_player_df = pd.merge(skater_EV_totals, skater_PP_totals, how="left", on=["Player", "Player"])
#combined_player_df

In [30]:
combined_player_df = combined_player_df.drop(columns = ['GF%_x','GF_x','GA_x','Season_y','Team_y','Position_y','GP_y','GF%_y','GF_y','GA_y','G+/-_y'])
#combined_player_df

In [31]:
#Merge EV,PP,SH tables together
full_player_totals = pd.merge(combined_player_df, skater_SH_totals, how="left", on=["Player", "Player"])
#full_player_totals.head(10)

In [32]:
#Clean up data
full_player_totals = full_player_totals.drop(columns = ['Season','Team','Position','GP','GF%','GF','GA','G+/-'])
#full_player_totals.head(10)

In [33]:
#Merge EV,PP,SH and Points tables together
full_player_totals = pd.merge(full_player_totals, skater_box_totals, how="left", on=["Player", "Player"])
#full_player_totals.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TOI_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [34]:
#Drop Repeated Columns
full_player_totals = full_player_totals.drop(columns = ['Season','Team','Position','GP','G','A1','A2'])
#full_player_totals.head(10)


In [35]:
#Rename Columns
full_player_totals = full_player_totals.rename(columns = {'Season_x':'Season','Team_x':'Team','Position_x':'Position','GP_x':'GP','TOI_x':'TOI_EV','G+/-_x':'G_EV','xG+/-':'xG_EV','TOI_x':'TOI_PP','GAA_x':'GAA_PP','TOI_x':'TOI_SH','GAA_y':'GAA_SH','TOI_y':'TOI'})
#full_player_totals

In [39]:
#Replace NaN with 0
full_player_totals = full_player_totals.replace(np.NaN, 0)
#full_player_totals

In [37]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Goalie%20Stats.csv')
#print(goalie_stats_raw)
#goalie_stats_raw.head()

In [40]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GA','Sv%','GSAx']]
#goalie_stats.head()

In [42]:
#Add Active / Inactive Column to both data tables
full_player_totals['Active/Inactive'] = 'Active'
full_player_totals

,Player,Season,Team,Position,GP,TOI_SH,xGF%,xGF,xGA,G_EV,xG_EV,TOI,GAA_PP,TOI_SH,GAA_SH,TOI,Points,PAA,Active/Inactive
0,A.J. Greer,21-22,N.J,L,9,70.45,58.74,3.16,2.22,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-20.633276,Active
1,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-35.092102,1537.92,41,26.890075,Active
2,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-35.092102,878.18,22,7.890075,Active
3,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,134.87,-35.092102,1519.70,57,42.890075,Active
4,Aaron Ekblad,19-20,FLA,D,67,1220.55,50.83,49.88,48.26,16.49,1.62,95.90,-2.792102,86.47,-19.232102,1537.92,41,26.890075,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29852,Zemgus Girgensons,19-20,BUF,C,69,783.73,49.54,25.39,25.86,-4.34,-0.47,54.37,-5.272102,81.28,-26.382102,828.20,18,-4.633276,Active
29853,Zemgus Girgensons,21-22,BUF,C/L,56,668.30,45.90,20.39,24.03,-5.14,-3.64,54.37,-5.272102,147.87,-34.932102,951.77,19,-3.633276,Active
29854,Zemgus Girgensons,21-22,BUF,C/L,56,668.30,45.90,20.39,24.03,-5.14,-3.64,54.37,-5.272102,147.87,-34.932102,828.20,18,-4.633276,Active
29855,Zemgus Girgensons,21-22,BUF,C/L,56,668.30,45.90,20.39,24.03,-5.14,-3.64,54.37,-5.272102,81.28,-26.382102,951.77,19,-3.633276,Active


In [46]:
#Make a Data Table for each team
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters

,Player,Season,Team,Position,GP,TOI_SH,xGF%,xGF,xGA,G_EV,xG_EV,TOI,GAA_PP,TOI_SH,GAA_SH,TOI,Points,PAA,Active/Inactive
1221,Alex Newhook,20-21,COL,C,6,74.70,67.70,4.38,2.09,5.32,2.29,97.58,-3.652102,0.00,0.000000,82.07,3,-19.633276,Active
1222,Alex Newhook,20-21,COL,C,6,74.70,67.70,4.38,2.09,5.32,2.29,97.58,-3.652102,0.00,0.000000,963.07,33,10.366724,Active
1223,Alex Newhook,21-22,COL,C,71,842.32,50.05,32.05,31.98,12.63,0.07,97.58,-3.652102,0.00,0.000000,82.07,3,-19.633276,Active
1224,Alex Newhook,21-22,COL,C,71,842.32,50.05,32.05,31.98,12.63,0.07,97.58,-3.652102,0.00,0.000000,963.07,33,10.366724,Active
1662,Andre Burakovsky,19-20,COL,L,58,707.17,49.18,29.04,30.01,14.44,-0.97,136.92,-1.782102,0.00,0.000000,881.17,45,22.366724,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28834,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-24.962102,134.62,6,-16.633276,Active
28835,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-24.962102,816.87,25,2.366724,Active
28836,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-24.962102,856.40,17,-5.633276,Active
28837,Vladislav Namestnikov,19-20,COL,C,9,118.45,56.20,5.39,4.20,3.42,1.19,44.83,-12.232102,97.72,-24.962102,184.92,5,-17.633276,Active


In [ ]:
#Inspect Data
full_player_totals

In [ ]:
#skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/PP%20On-Ice%20Rates.csv')
#print(skater_PP_rates_raw)
#skater_PP_rates_raw.head()

**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [ ]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_player_totals.columns

Index(['Player', 'Season', 'Team', 'Position', 'GP', 'TOI_SH', 'xGF%', 'xGF',
       'xGA', 'G_EV', 'xG_EV', 'TOI', 'GAA_PP', 'TOI_SH', 'GAA_SH', 'TOI',
       'Points', 'PAA'],
      dtype='object')

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_player_totals dataframe and the goalie_stats dataframe.

Add folder in github for the csv's, update github links in code to pull data